# Inicialização

In [3]:
import tensorflow as tf

2022-04-28 23:48:49.525322: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-28 23:48:49.526109: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
print(tf.__version__)

2.7.0


# Conectando com servidor

In [15]:
import requests
import pandas as pd
import numpy as np

In [11]:
url = "http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploTreinoAcaos/obtemParaTeste"

In [12]:
r = requests.get(url)
j = r.json()
df = pd.DataFrame.from_dict(j)

In [13]:
def trataLinhaX(linha):
    return np.fromstring(linha['campoX'], dtype=float, sep=',')

In [16]:
dfTratadoX = df.apply(trataLinhaX, axis='columns')

In [18]:
X = np.stack(dfTratadoX.values)

In [19]:
X.shape

(3, 1800)

In [20]:
Y = np.stack(df['campoY'].values)

In [21]:
Y = Y.reshape(3,1)

In [22]:
Y.shape

(3, 1)

# Minha Rede Simples

In [54]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(1800,)),
    tf.keras.layers.Dense(900, activation=tf.nn.relu), 
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])


In [55]:
model.build()

In [56]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 900)               1620900   
                                                                 
 dense_17 (Dense)            (None, 10)                9010      
                                                                 
Total params: 1,629,910
Trainable params: 1,629,910
Non-trainable params: 0
_________________________________________________________________


In [57]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [58]:
history = model.fit(x=X,y=Y, epochs=10)

Epoch 1/10
1/1 [==============================] - 6s 6s/step - loss: 2.1793 - accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 98ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - 0s 26ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 0s 171ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 0s 32ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 28ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 0s 35ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/10
1/1 [==============================] - 0s 31ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/10
1/1 [==============================] - 0s 29ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 10/10
1/1 [==============================] - 0s 40ms/step - loss: 0.0000e+00 - accurac